# Purpose

Find a way to get data from Plugshare.com since they're not responding to my API access request. The comments and metadata from stations across different networks should be extremely useful in diagnosing electrical and non-electrical customer experience issues.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from rich import print
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests

from evlens.data.plugshare import Scraper

from dotenv import load_dotenv
load_dotenv(override=True)

from evlens.logs import setup_logger
logger = setup_logger("Notebook-0.1")
logger.info("TEST!")

2024-06-16_T14_49_14EDT: INFO (Notebook-0.1:L18) - TEST!


# Testing our custom scraper

In [2]:
# Electrify America in Springfield, VA mall parking lot
TEST_LOCATION = 252784

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Electrify America in Springfield, VA mall parking lot
TEST_LOCATION = 252784
url = f"https://www.plugshare.com/location/{TEST_LOCATION}"

chrome_options = Options()
# chrome_options.add_argument('--headless=new')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_argument("--disable-extensions")

driver = webdriver.Chrome(options=chrome_options)

driver.get(url)

wait = WebDriverWait(driver, 5)

# Wait for the cookie dialog to appear
try:
    iframe = wait.until(EC.visibility_of_element_located((
        By.ID,
        "global-consent-notice"
    )))
    logger.info("Found the banner!")
    driver.switch_to.frame(iframe)
    
except (NoSuchElementException, TimeoutException) as e1:
    logger.warning("No cookie dialog iframe found, moving on!")

2024-06-16_T15_25_30EDT: INFO (Notebook-0.1:L29) - Found the banner!


In [6]:
# Can I find the Accept button?
main_buttons = driver.find_elements(By.TAG_NAME, "button")
len(main_buttons)

1

*Note that the only button is the Accept one, we need another mechanism for finding the Manage Settings link...*

In [24]:
<iframe src="https://cmp-consent-tool.privacymanager.io/latest/index.html#/notice?theme=defaultTheme&amp;useSystemFonts=false&amp;cmpType=global" id="global-consent-notice" aria-label="Privacy Manager window." role="dialog" aria-modal="true" frameborder="0" scrolling="yes" style="min-height: 72px; max-height: 105px; width: 100%; left: 0px; bottom: 0px; position: fixed; display: block; z-index: 2147483647; box-shadow: rgba(0, 0, 0, 0.12) 0px 6px 30px 5px, rgba(0, 0, 0, 0.14) 0px 16px 24px 2px, rgba(0, 0, 0, 0.2) 0px 8px 10px -5px; height: 105px;" cd_frame_id_="78387f53d0664ffe5d1b4116ea60ee08"></iframe>

In [29]:
# Example: Find a link with class "my-link" and href attribute starting with "https://..."
manage_cookies_link = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Customize your consent preferences.']") 
manage_cookies_link

<selenium.webdriver.remote.webelement.WebElement (session="fbd40d5ce7cf519b6687dce30e11487a", element="f.87B02411C39E39D044C9D4C04D7BCF73.d.38D20518693AD9A1F13EDE88F3C4577B.e.61")>

In [30]:
manage_cookies_link.click()

In [34]:
reject_all_button = driver.find_element(By.CSS_SELECTOR, "button[id='denyAll']")
reject_all_button

<selenium.webdriver.remote.webelement.WebElement (session="fbd40d5ce7cf519b6687dce30e11487a", element="f.87B02411C39E39D044C9D4C04D7BCF73.d.38D20518693AD9A1F13EDE88F3C4577B.e.374")>

In [35]:
reject_all_button.click()

In [37]:
reject_all_button_confirm = driver.find_element(By.XPATH, "//*[@id=\"mat-dialog-0\"]/ng-component/app-theme/div/div/div[2]/button[2]")
reject_all_button_confirm

<selenium.webdriver.remote.webelement.WebElement (session="fbd40d5ce7cf519b6687dce30e11487a", element="f.87B02411C39E39D044C9D4C04D7BCF73.d.38D20518693AD9A1F13EDE88F3C4577B.e.648")>

In [38]:
reject_all_button_confirm.click()

In [39]:
# Switch back to main frame
driver.switch_to.default_content()

# Exit login dialog
try:
    # Wait for the exit button
    wait = WebDriverWait(driver, 1)
    esc_button = wait.until(EC.visibility_of_element_located((
        By.XPATH,
        # "//*[@id=\"dialogContent_authenticate\"]/button/md-icon" # old
        "//*[@id=\"dialogContent_authenticate\"]/button" # from chrome
    )))
    esc_button.click()
    logger.info("Found the login escape button and clicked it!")

except (NoSuchElementException, TimeoutException):
    logger.error("Login dialog exit button not found.")

except Exception as e:
    logger.error(f"Unknown error trying to exit login dialog: {e}")

2024-06-16_T15_31_11EDT: INFO (Notebook-0.1:L14) - Found the login escape button and clicked it!


In [40]:
driver.quit()

In [ ]:
print('a')

In [29]:
# Do a single element scrape
try: ## FIND STATION NAME
    wait.until(EC.visibility_of_element_located((
        By.XPATH,
        "//*[@id=\"display-name\"]/div/h1"
    )))
    name = driver.find_element(
        By.XPATH,
        "//*[@id=\"display-name\"]/div/h1"
        ).text
except:
    logger.error("Station name error", exc_info=True)
    name = np.nan
    
name

2024-06-14_T09_24_24EDT: ERROR (Notebook-0.1:L12) - Station name error
Traceback (most recent call last):
  File "/var/folders/98/fhwnl49n19l_xywxzghbm4jm0000gn/T/ipykernel_58677/2682536154.py", line 3, in <module>
    wait.until(EC.visibility_of_element_located((
  File "/Users/davemcrench/Documents/Projects/evlens/.venv/lib/python3.11/site-packages/selenium/webdriver/support/wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



nan

In [25]:
driver.quit()

In [28]:
# Why is it taking so long to even *start* trying to exit the login dialog?!
s = Scraper(timeout=3)

# Scrape only one location that I can test via browser
df = s.scrape_plugshare_locations(TEST_LOCATION, TEST_LOCATION)
df.info()
df.head()

In [19]:
df.loc[0, 'Comments']

'J-1772 1 Plug 7 kW\n1 Station\nElectrify America, CCS/SAE 3 Plugs 350 kW\n3 Stations\nElectrify America, , Jun 7, 2024\ncheck_circleJ C\nBMW iX 2024, Jun 5, 2024\ncheck_circleKJva\nMercedes EQE 350350 Kilowatts, May 30, 2024\ncheck_circleKJva\nMercedes EQE 350350 Kilowatts, May 24, 2024\ncheck_circleBEV\nChevrolet Blazer EV 2024\nSo far so good! Screen is cracked on the charger, difficult to read the prompts… but I guess it doesn’t matter if it charges!, May 24, 2024\ncheck_circleKMac\nKia EV6 2022235 Kilowatts\nScreen on 1 not working but can charge through the app, 0.08 mi\n9.3 Springfield Town Center - Target - East Lot (2)\nJ-1772, 0.18 mi\n4.8 Springfield Town Center - LA Fitness\nJ-1772, 0.22 mi\n1 Springfield Town Center - Frontier Garage\nJ-1772, 0.22 mi\n10 Springfield Town Center - Target - West Lot\nNACS (Tesla), 0.23 mi\n4.7 Springfield Town Center - Maggianos\nJ-1772, , , '